In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import scipy.stats as stats
from datetime import timedelta, datetime
import os
import seaborn as sns


In [16]:
DATA_PATH = '/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/p_data/p_einzelnt/'
MONTHS = ["p01", "p02", "p03", "p04", "p05", "p06", "p07", "p08", "p09", "p10", "p11", "p12"]


In [17]:
def find_closest_sensor(incidence_df, sensor_df, max_distance=0.3, mainline_threshold=0.3): 
    # Sortiere Sensoren für schnelleres Suchen
    sensor_df = sensor_df.sort_values(by=['Fwy', 'Abs PM'])
    closest_sensors = []

    for _, incident in incidence_df.iterrows():
        fwy = incident['Fwy']
        abs_pm = incident['Abs PM']

        # Suche HOV-Sensor
        hov_sensors = sensor_df[(sensor_df['Fwy'] == fwy) & (sensor_df['Type'] == 'HOV')]
        hov_preceding = hov_sensors[
            (hov_sensors['Abs PM'] <= abs_pm) & 
            (abs_pm - hov_sensors['Abs PM'] <= max_distance)
        ]
        if hov_preceding.empty:
            continue
        hov_sensor = hov_preceding.iloc[-1]

        # Suche Mainline-Sensor im selben Fwy
        mainline_sensors = sensor_df[(sensor_df['Fwy'] == fwy) & (sensor_df['Type'] == 'mainline')]
        if mainline_sensors.empty:
            continue
        
        # Ermittle Abstand vom HOV-Sensor
        mainline_sensors = mainline_sensors.copy()
        mainline_sensors['distance_to_hov'] = abs(mainline_sensors['Abs PM'] - hov_sensor['Abs PM'])
        
        # Filter: max. mainline_threshold Meilen Unterschied
        mainline_candidates = mainline_sensors[mainline_sensors['distance_to_hov'] <= mainline_threshold]
        if mainline_candidates.empty:
            continue

        # Nimm den nächstgelegenen Mainline-Sensor
        mainline_sensor = mainline_candidates.iloc[0]

        closest_sensors.append({
            'Incident_ABS_PM': abs_pm,
            'Incident_Fwy': fwy,
            "hov_station_id": hov_sensor['station_id'], 
            'HOV_Sensor_ABS_PM': hov_sensor['Abs PM'],
            'mainline_station_id': mainline_sensor['station_id'],
            'Mainline_Sensor_ABS_PM': mainline_sensor['Abs PM'],
            'time': incident['dt'],
            'Distance': abs_pm - hov_sensor['Abs PM']
        })

    return pd.DataFrame(closest_sensors)

In [18]:
def datetime_to_timeslot(dt, month_start):
    """Konvertiert datetime zu einem Timeslot-Index relativ zum Monatsanfang"""
    delta = dt - month_start
    return int(delta.total_seconds() // 300)


In [19]:
# Lade Vorfall-Daten
incidents = pd.read_csv('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/incidents.csv', 
                         sep="\t", parse_dates=['dt'])
incidents['month'] = incidents['dt'].dt.to_period('M')

# Filter: Nur Unfälle und solche mit Duration >= 15 Minuten
incidents = incidents[incidents['type'] == "accident"]
incidents = incidents[incidents['Duration (mins)'] >= 15]

# Lade Sensor-Metadaten
sensor_meta = pd.read_csv('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/sensor_meta_feature.csv', 
                          sep="\t")

In [ ]:
# Hole alle DESCRIPTION-Werte (z.B. "1144-Fatality", "1181-Trfc Collision-Minor Inj", …)
descriptions = incidents['DESCRIPTION'].unique()
all_results = {}

# Für jede DESCRIPTION erfolgt eine separate Analyse
for desc in descriptions:
    print(f"\n--- Analyse für DESCRIPTION: {desc} ---")
    # Filtere Unfälle für die aktuelle DESCRIPTION
    desc_incidents = incidents[incidents['DESCRIPTION'] == desc]
    all_data_desc = []
    
    # Gruppiere nach Monat
    for month, month_incidents in desc_incidents.groupby('month'):
        month_str = f"p{month.month:02d}"
        if month_str not in MONTHS:
            continue
        
        try:
            # Lade Verkehrsdaten und zugehörige station_ids (hier gilt für beide Sensortypen)
            traffic_data = np.load(os.path.join(DATA_PATH, f'{month_str}_car.npy'))
            station_ids = np.load(os.path.join(DATA_PATH, f'{month_str}_car_node.npy')).astype(str)
        except FileNotFoundError:
            print(f"Dateien für {month_str} nicht gefunden")
            continue
        
        station_to_index = {sid: i for i, sid in enumerate(station_ids)}
        
        # Finde für den Monat die Vorfälle mit passender Sensor-Kombination (HOV + Mainline)
        closest_sensors = find_closest_sensor(month_incidents, sensor_meta)
        if closest_sensors.empty:
            continue
        
        # Mapping: Hole den Mainline-Sensor-Index
        closest_sensors['mainline_sensor_index'] = closest_sensors['mainline_station_id'].astype(str).map(station_to_index)
        valid_sensors = closest_sensors.dropna(subset=['mainline_sensor_index']).copy()
        
        # Zeitkonvertierung: Verschiebe Zeitpunkt um 5 Minuten zurück und berechne Timeslot
        month_start = datetime(month.year, month.month, 1)
        valid_sensors['time'] = pd.to_datetime(valid_sensors['time'], errors='coerce') - pd.Timedelta(minutes=5)
        valid_sensors['timeslot'] = valid_sensors['time'].apply(lambda x: datetime_to_timeslot(x, month_start))
        
        time_window = 3  # z. B. 3 Intervalle vor und nach dem Ereignis (jeweils 5 Minuten)
        for _, row in valid_sensors.iterrows():
            idx = int(row['mainline_sensor_index'])  # Hier werden die Daten des Mainline-Sensors ausgewertet
            ts = row['timeslot']
    
            if ts < 0 or ts >= traffic_data.shape[0]:
                continue
            
            start = max(0, ts - time_window)
            end = min(traffic_data.shape[0], ts + time_window + 1)  # +1 für inklusiven Schnitt
            
            if (end - start) < (2 * time_window):
                continue  # Überspringe, falls das Zeitfenster zu klein ist
                
            traffic_slice = traffic_data[start:end, idx]
            if np.isnan(traffic_slice).any():
                continue  # Überspringe, wenn NaNs enthalten sind
                
            all_data_desc.append({
                'pre': traffic_slice[:time_window],
                'post': traffic_slice[time_window+1:],  # Unfallzeitpunkt überspringen
                'feature_names': ['Traffic Volume', 'Occupancy Rate', 'Speed']
            })
        print(f"{month_str} für DESCRIPTION {desc} verarbeitet")
    
    if len(all_data_desc) == 0:
        print(f"Keine gültigen Daten für DESCRIPTION {desc}")
        continue
    
    # Aggregiere Pre- und Post-Daten aus allen gültigen Vorfällen
    pre_data = []
    post_data = []
    for entry in all_data_desc:
        pre_data.extend(entry['pre'][:])
        post_data.extend(entry['post'][:])
    
    # Optional: Entferne die letzten 3 Elemente aus pre_data, falls vorhanden
    if len(pre_data) > 3:
        pre_data.pop()
        pre_data.pop()
        pre_data.pop()
    
    min_length = min(len(pre_data), len(post_data))
    if min_length == 0:
        print(f"Nicht genügend Daten für DESCRIPTION {desc}")
        continue
    
    stat, p_value = stats.wilcoxon(pre_data[:min_length], post_data[:min_length], alternative='greater')
    feature_name = 'Traffic Volume'
    all_results[desc] = {
        'statistic': stat,
        'p_value': p_value,
        'median_pre': np.median(pre_data[:min_length]),
        'median_post': np.median(post_data[:min_length])
    }
    
    # Daten in numpy-Arrays und Kappen von Extremwerten (z.B. 99%-Quantil)
    pre_data = np.array(pre_data[:min_length])
    post_data = np.array(post_data[:min_length])
    pre_data = np.clip(pre_data, 0, np.percentile(pre_data, 99))
    post_data = np.clip(post_data, 0, np.percentile(post_data, 99))
    
    # --- Boxplot Visualisierung ---
    plt.figure(figsize=(8, 6))
    plt.ylim(0, 500)
    sns.boxplot(data=[pre_data, post_data], palette=["#4c72b0", "#dd8452"])
    plt.xticks([0, 1], ['Pre-Incident', 'Post-Incident'])
    plt.title(f"Traffic Volume (Mainline) für {desc}\np-value: {p_value:.4f}")
    plt.ylabel('Traffic Volume (Normalized)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()
    
    # --- Violinplot Visualisierung ---
    data_plot = pd.DataFrame({
        'Traffic Volume': np.concatenate([pre_data, post_data]),
        'Period': ['Pre-Incident'] * len(pre_data) + ['Post-Incident'] * len(post_data)
    })
    plt.figure(figsize=(10, 6))
    sns.violinplot(x='Period', y='Traffic Volume', data=data_plot, palette=["#4c72b0", "#dd8452"])
    plt.title(f'Verteilung des Mainline Traffic Volume für {desc}')
    plt.xlabel('')
    plt.ylabel('Traffic Volume (Normalized)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

# Ausgabe der statistischen Ergebnisse pro DESCRIPTION
print("\n=== Statistische Ergebnisse pro DESCRIPTION ===")
for desc, res in all_results.items():
    print(f"\n{desc}:")
    print(f"  Wilcoxon-Statistik: {res['statistic']}")
    print(f"  P-Wert: {res['p_value']:.4f}")
    print(f"  Median vorher: {res['median_pre']:.2f}")
    print(f"  Median nachher: {res['median_post']:.2f}")
    print(f"  Signifikant (p < 0.05): {'Ja' if res['p_value'] < 0.05 else 'Nein'}")



--- Analyse für DESCRIPTION: 20002-Hit and Run No Injuries ---
Keine gültigen Daten für DESCRIPTION 20002-Hit and Run No Injuries

--- Analyse für DESCRIPTION: 1182-Trfc Collision-No Inj ---
Keine gültigen Daten für DESCRIPTION 1182-Trfc Collision-No Inj

--- Analyse für DESCRIPTION: 1183-Trfc Collision-Unkn Inj ---
